# Comparing LLM Path Extractors for Knowledge Graph Construction

In this notebook, we'll compare three different LLM Path Extractors from llama_index:
1. SimpleLLMPathExtractor
2. SchemaLLMPathExtractor
3. DynamicLLMPathExtractor (New)

We'll use a Wikipedia page as our test data and visualize the resulting knowledge graphs using Pyvis.

## Setup and Imports

In [ ]:
# !pip install llama_index pyvis wikipedia

In [ ]:
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

import wikipedia

import os

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Set up LLM Backend

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."

# Set up global configurations
llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

Settings.llm = llm
Settings.chunk_size = 2048
Settings.chunk_overlap = 20

## Fetch Some Raw Text from Wikipedia

In [ ]:
def get_wikipedia_content(title):
    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation page. Options: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"Page '{title}' does not exist.")
    return None

In [ ]:
wiki_title = "Barack Obama"
content = get_wikipedia_content(wiki_title)

if content:
    document = Document(text=content, metadata={"title": wiki_title})
    print(f"Fetched content for '{wiki_title}' (length: {len(content)} characters)")
else:
    print("Failed to fetch Wikipedia content.")

Fetched content for 'Barack Obama' (length: 84362 characters)


## 1. SimpleLLMPathExtractor

In [ ]:
kg_extractor = SimpleLLMPathExtractor(llm=llm, max_paths_per_chunk=20, num_workers=4)

simple_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

simple_index.property_graph_store.save_networkx_graph(name="./SimpleGraph.html")
simple_index.property_graph_store.get_triplets(entity_names=["Barack Obama", "Obama"])[
    :5
]

Extracting paths from text: 100%|██████████| 11/11 [00:09<00:00,  1.12it/s]


[(EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'db15556a-d04b-480f-8e73-01b0d25ff950'}, name='Obama'),
  Relation(label='Selected as', source_id='Obama', target_id='Editor of harvard law review', properties={'title': 'Barack Obama', 'triplet_source_id': '9adfab03-5b81-4dfe-b655-06173e69ddf1'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '9adfab03-5b81-4dfe-b655-06173e69ddf1'}, name='Editor of harvard law review')),
 (EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'db15556a-d04b-480f-8e73-01b0d25ff950'}, name='Obama'),
  Relation(label='Visited', source_id='Obama', target_id='White house', properties={'title': 'Barack Obama', 'triplet_source_id': 'b31c90aa-5fd3-4f90-bbfa-df187d9936ca'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'b31c90aa-5fd3-4f90-bbfa-df187d

## 2. DynamicLLMPathExtractor

### Without intial ontology :
Here, we let the LLM define the ontology on the fly, giving it full freedom to label the nodes as it best sees fit.

In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    # Let the LLM infer entities and their labels (types) on the fly
    allowed_entity_types=None,
    # Let the LLM infer relationships on the fly
    allowed_relation_types=None,
    # LLM will generate any entity properties, set `None` to skip property generation (will be faster without)
    allowed_relation_props=[],
    # LLM will generate any relation properties, set `None` to skip property generation (will be faster without)
    allowed_entity_props=[],
)

dynamic_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index.property_graph_store.save_networkx_graph(name="./DynamicGraph.html")

dynamic_index.property_graph_store.get_triplets(entity_names=["Barack Obama", "Obama"])[
    :5
]

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 49.17it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:40<00:00,  3.69s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '29cd5c21-f3d6-4ee4-96d8-e90f5b3c6568'}, name='Barack Obama'),
  Relation(label='INCLUDES', source_id='Barack Obama', target_id='ACA', properties={'provisions': 'health-related', 'title': 'Barack Obama', 'triplet_source_id': '7476d205-e40d-4bed-9c5d-acb63c256121'}),
  EntityNode(label='POLICY', embedding=None, properties={'provisions': 'health-related', 'title': 'Barack Obama', 'triplet_source_id': '7476d205-e40d-4bed-9c5d-acb63c256121'}, name='ACA')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '29cd5c21-f3d6-4ee4-96d8-e90f5b3c6568'}, name='Barack Obama'),
  Relation(label='SIGNED', source_id='Barack Obama', target_id='Budget Control Act of 2011', properties={'date': 'August 2, 2011', 'title': 'Barack Obama', 'triplet_source_id': '54ddec15-b4ee-45b2-ad06-06e7eda9f356'}),
  EntityNode(label='LEGISLATION', embedding=None, prope

### With initial ontology for guided KG extraction : 
Here, we have partial knowledge of what we want to detect, we know the article is about Barack Obama, so we define some entities and relations that could help guide the LLM in the labeling process as it detects the entities and relations. This doesn't guarantee that the LLM will use them, it simply guides it and gives it some ideas. It will still be up to the LLM to decide whether it uses the entities and relations we provide or not. 

In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=["POLITICIAN", "POLITICAL_PARTY"],
    allowed_relation_types=["PRESIDENT_OF", "MEMBER_OF"],
    allowed_relation_props=["description"],
    allowed_entity_props=["description"],
)

dynamic_index_2 = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index_2.property_graph_store.save_networkx_graph(name="./DynamicGraph_2.html")
dynamic_index_2.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 45.20it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:45<00:00,  4.09s/it]


[(EntityNode(label='POLITICIAN', embedding=None, properties={'description': '44th President of the United States', 'title': 'Barack Obama', 'triplet_source_id': '531221eb-d0b0-449c-acc7-f5696ceaf58c'}, name='Barack Obama'),
  Relation(label='IMPLEMENTS', source_id='Barack Obama', target_id='taxes on insurance companies that offer expensive plans', properties={'title': 'Barack Obama', 'triplet_source_id': '914a08ce-7915-44cc-9a52-d661370d5a73'}),
  EntityNode(label='TAX', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '914a08ce-7915-44cc-9a52-d661370d5a73'}, name='taxes on insurance companies that offer expensive plans')),
 (EntityNode(label='POLITICIAN', embedding=None, properties={'description': '44th President of the United States', 'title': 'Barack Obama', 'triplet_source_id': '531221eb-d0b0-449c-acc7-f5696ceaf58c'}, name='Barack Obama'),
  Relation(label='SIGNED', source_id='Barack Obama', target_id='Budget Control Act of 2011', properties={'date': 'Augus

# 3 - SchemaLLMPathExtractor

In [ ]:
kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    strict=False,  # Set to False to showcase why it's not going to be the same as DynamicLLMPathExtractor
    possible_entities=None,  # USE DEFAULT ENTITIES (PERSON, ORGANIZATION... etc)
    possible_relations=None,  # USE DEFAULT RELATIONSHIPS
    possible_relation_props=[
        "extra_description"
    ],  # Set to `None` to skip property generation
    possible_entity_props=[
        "extra_description"
    ],  # Set to `None` to skip property generation
    num_workers=4,
)

schema_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

schema_index.property_graph_store.save_networkx_graph(name="./SchemaGraph.html")
schema_index.property_graph_store.get_triplets(entity_names=["Barack Obama", "Obama"])[
    :5
]


Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 41.31it/s]












Extracting paths from text with schema: 100%|██████████| 11/11 [02:02<00:00, 11.14s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'e3199329-841e-4080-ab5a-43dbcf2f6a8e'}, name='Barack Obama'),
  Relation(label='HAS', source_id='Barack Obama', target_id='economic stimulus package', properties={'extra_description': 'Legacy and recognition', 'title': 'Barack Obama', 'triplet_source_id': 'e3199329-841e-4080-ab5a-43dbcf2f6a8e'}),
  EntityNode(label='CONCEPT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'e3199329-841e-4080-ab5a-43dbcf2f6a8e'}, name='economic stimulus package')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'e3199329-841e-4080-ab5a-43dbcf2f6a8e'}, name='Barack Obama'),
  Relation(label='ACKNOWLEDGED', source_id='Barack Obama', target_id='Youthful image', properties={'extra_description': 'Acknowledged his youthful image', 'title': 'Barack Obama', 'triplet_source_id': '0fb0c21c-cd37-431c-ac8f-f6643e0d4c59'}),
  Entity

## Comparison and Analysis

Let's compare the results of the three extractors:

1. **SimpleLLMPathExtractor**: This extractor creates a basic knowledge graph without any predefined schema. It may produce a larger number of diverse relationships but might lack consistency in entity and relation naming.


3. **DynamicLLMPathExtractor**: 
    - This new extractor combines the flexibility of SimpleLLMPathExtractor with some initial guidance from a schema. It can expand beyond the initial entity and relation types, potentially producing a rich and diverse graph while maintaining some level of consistency. 
    - Not giving it any entities or relations to start with in the input gives the LLM complete freedom to infer the schema on the fly as it best sees fit. This is going to vary based on the LLM and the temperature used.

3. **SchemaLLMPathExtractor**: With a predefined schema, this extractor produces a more structured graph. The entities and relations are limited to those specified in the schema, which can lead to a more consistent but potentially less comprehensive graph. Even if we set "strict" to false, the extracted KG Graph doesn't reflect the LLM's pursuit of trying to find new entities and types that fall outside of the input schema's scope.


## Key observations:

- The SimpleLLMPathExtractor graph might have the most diverse set of entities and relations.
- The SchemaLLMPathExtractor graph should be the most consistent but might miss a lot of relationships that don't fit the predefined schema, even if we don't impose a strict validation of the schema.
- The DynamicLLMPathExtractor graph should show a balance between diversity and consistency, potentially capturing important relationships that the schema-based approach might miss while still maintaining some structure.

## The choice between these extractors depends on the specific use case:

- Use SimpleLLMPathExtractor for exploratory analysis where you want to capture a wide range of potential relationships for RAG applications, without caring about the entity types.
- Use SchemaLLMPathExtractor when you have a well-defined domain and want to ensure consistency in the extracted knowledge.
- Use DynamicLLMPathExtractor when you want a balance between structure and flexibility, allowing the model to discover new entity and relation types while still providing some initial guidance. This one is especially useful if you want a KG with labeled (typed) entities but don't have an input Schema (or you've partially defined the schema as a starting base).